In [19]:
import numpy as np
import os
from astroquery.jplhorizons import Horizons

In [20]:
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [21]:
# File taken from columns in OG spreadsheet WITH < 3AU OBS REMOVED**
# (just did this in an excel spreadsheet)
lc_file = path_67p+'LSST_3AU_sun_comet.dat'

In [22]:
mjd = []
sun_x = []; sun_y = []; sun_z = []

f = open(lc_file, 'r')
lines = f.readlines()
for line in lines:
    data=line.split()
    mjd.append(float(data[0]))
    sun_x.append(float(data[2]))
    sun_y.append(float(data[3]))
    sun_z.append(float(data[4]))

In [23]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function

# Splitting this into 2 because Horizons wouldn't let me do all 655 epochs at once :(
obj = Horizons(id='90000693', location='500',epochs=mjd[:500])

In [25]:
earth_x = []; earth_y=[]; earth_z=[];
vec = obj.vectors(refplane='earth')  # LSST output in equatorial ref frame!!!
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [26]:
# Check output looks sensible
vec

targetname,datetime_jd,datetime_str,M1,M2,k1,k2,phasecoeff,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,mag,---,---,mag / deg,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2459953.863937,A.D. 2023-Jan-09 08:44:04.1568,11.0,13.9,12.25,5.0,0.03,-3.444507550839725,-2.015299434893513,-0.7824767900472543,0.0136151232058168,-0.001640122462597969,-0.001023023336391795,0.02348750285887415,4.066735054934935,-0.01052233898239546
67P/Churyumov-Gerasimenko,2459956.868077,A.D. 2023-Jan-12 08:50:01.8528,11.0,13.9,12.25,5.0,0.03,-3.403980486305475,-2.019010618092604,-0.7850255482630261,0.0133581633439457,-0.0008328123718247615,-0.000674750606178298,0.02330316234953832,4.034817485159,-0.01072161851896468
67P/Churyumov-Gerasimenko,2459957.822871,A.D. 2023-Jan-13 07:44:56.0544,11.0,13.9,12.25,5.0,0.03,-3.391269338533897,-2.019684551018559,-0.785617485401405,0.01326702018707859,-0.0005791128586201759,-0.0005652774029997287,0.02324388062877702,4.024553173393488,-0.01077841931749734
67P/Churyumov-Gerasimenko,2459964.842786,A.D. 2023-Jan-20 08:13:36.7104,11.0,13.9,12.25,5.0,0.03,-3.300841541600359,-2.017324421929434,-0.7868109885510691,0.01245370106999689,0.001235311086855243,0.0002185458387100065,0.02279993248169711,3.947685934540927,-0.01108793152644783
67P/Churyumov-Gerasimenko,2459966.871729,A.D. 2023-Jan-22 08:55:17.3856,11.0,13.9,12.25,5.0,0.03,-3.275857081570041,-2.014305209459407,-0.7861457359036982,0.01217081269076075,0.001738939524077199,0.0004364083264157982,0.02266969315223143,3.925135693680852,-0.01113736742887081
67P/Churyumov-Gerasimenko,2459966.876288,A.D. 2023-Jan-22 09:01:51.2832,11.0,13.9,12.25,5.0,0.03,-3.275801596337988,-2.014297279084376,-0.7861437452150545,0.01217015330572303,0.001740058064634656,0.0004368922359865737,0.0226693998976264,3.925084918219985,-0.01113745635090277
67P/Churyumov-Gerasimenko,2459970.830448,A.D. 2023-Jan-26 07:55:50.7072,11.0,13.9,12.25,5.0,0.03,-3.228859618664961,-2.005530730112153,-0.7836003814604762,0.01156078853651734,0.002685690649930958,0.00084585838074295,0.02241445664734323,3.880942862796929,-0.01117697808981592
67P/Churyumov-Gerasimenko,2459970.830859,A.D. 2023-Jan-26 07:56:26.2176,11.0,13.9,12.25,5.0,0.03,-3.228854867194669,-2.005529626273644,-0.7836000338041871,0.0115607215126621,0.002685786301514825,0.0008458997229605441,0.02241443011612507,3.880938269058905,-0.01117697834034203


In [27]:
# Now get the remaining 655-500 vectors
obj = Horizons(id='90000693', location='500', epochs=mjd[500:])

In [28]:
vec = obj.vectors(refplane='earth')  
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [1]:
# Check this didn't miss anything or double anything
#earth_x[495:]

In [109]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])

[-3.75625676 -1.15950396 -0.41150245]


In [111]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [30]:
# This is just to grab ephemeris data to check phase angles
obj = Horizons(id='90000693', location='geocentric', epochs=mjd[:500])
ephem_table = obj.ephemerides(quantities='43')

In [31]:
#ephem_table

In [32]:
alpha_list = []
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [33]:
obj = Horizons(id='90000693', location='geocentric', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [34]:
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [37]:
#alpha_list

In [39]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_LSST_3AU_intensities.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
#intensities_list

In [ ]:
# Code to write to lightcurve file in Mikko format
# Uncomment fouts/change to print if necessary

#fout = open(path_67p+'67P_LC_LSST_3AU_NU.dat', 'w+')
sameday = []
fout.write('314\n')
for i, day in enumerate(mjd[:-1]):
    newday = True
    sameday.append(day)
    if abs(day-mjd[i+1])<0.5:
        newday = False
    if newday==False:
        if day not in sameday:
            sameday.append(day)
    else:
        #fout.write('%i 1\n'%(len(sameday)))
        for time in sameday:
            loc = mjd.index(time)
            #fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,intensities_list[loc],sun_x[loc], sun_y[loc], sun_z[loc], earth_x[loc], earth_y[loc], earth_z[loc]))
        sameday = []
#fout.write('1 1\n')
#fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(mjd[-1]+2400000.5,intensities_list[-1],sun_x[-1], sun_y[-1], sun_z[-1], earth_x[-1], earth_y[-1], earth_z[-1]))

#fout.close()

In [ ]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty

corr_fac = 16    # from mag_from_phase_func py script

#fout = open(path_67p+'67p_20210109_I11_R.dat', 'w+')
for i,day in enumerate(mjd):
    magnitude = corr_fac -2.5*np.log10(intensities_list[i])
    #fout.write('%.6f %.4f 0.01\n'%(day+2400000.5, magnitude))
#fout.close()